<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [1]:
import pandas as pd
import tweepy

In [2]:
keys= pd.read_table("TW.txt", header=None, sep=',')
keys = dict(zip(keys[0],keys[1]))

In [3]:
#Autenticarnos
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [10]:
tweets = tweepy.Cursor(api.search,
                       q=['Huracan'],
                       lang='es',
                       geocode=f'4.60769,-74.11439,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [11]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogota')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f'https://twitter.com/i/web/status/{tweet.id}')
    
    t.append(temp)

In [12]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,RT @ELTIEMPO: 👉 Mientras la alerta por el paso...,2020-11-16 20:08:56,Bogota,,1328429868380516352,https://twitter.com/i/web/status/1328429868380...
1,RT @IDEAMColombia: El huracán #IOTA ahora es d...,2020-11-16 20:08:47,Bogota,"Cali, Colombia",1328429832057839618,https://twitter.com/i/web/status/1328429832057...
2,@rudolf_hommes @velascoluisf @teleislas Si en ...,2020-11-16 20:08:45,Bogota,"Tena, Cundinamarca",1328429823987998722,https://twitter.com/i/web/status/1328429823987...
3,RT @smilelalis: Que alguien le explique a los ...,2020-11-16 20:08:45,Bogota,"Medellín, Colombia",1328429823887351808,https://twitter.com/i/web/status/1328429823887...
4,RT @LaClavijo_: Colombia es al único país que ...,2020-11-16 20:08:37,Bogota,"Risaralda, Colombia",1328429790064484353,https://twitter.com/i/web/status/1328429790064...


In [13]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,RT @ELTIEMPO: 👉 Mientras la alerta por el paso...,2020-11-16 20:08:56,Bogota
1,RT @IDEAMColombia: El huracán #IOTA ahora es d...,2020-11-16 20:08:47,Bogota
2,@rudolf_hommes @velascoluisf @teleislas Si en ...,2020-11-16 20:08:45,Bogota
3,RT @smilelalis: Que alguien le explique a los ...,2020-11-16 20:08:45,Bogota
4,RT @LaClavijo_: Colombia es al único país que ...,2020-11-16 20:08:37,Bogota


In [14]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [15]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,RT @ELTIEMPO: 👉 Mientras la alerta por el paso...,2020-11-16 20:08:56,Bogota,RT 👉 Mientras la alerta por el paso del hurac...,[],[],[@ELTIEMPO:]
1,RT @IDEAMColombia: El huracán #IOTA ahora es d...,2020-11-16 20:08:47,Bogota,"RT El huracán ahora es de categoría 5, prese...",[],[#IOTA],[@IDEAMColombia:]
2,@rudolf_hommes @velascoluisf @teleislas Si en ...,2020-11-16 20:08:45,Bogota,"Si en condiciones relativamente ""normales"" ...",[],[],"[@rudolf_hommes, @velascoluisf, @teleislas]"
3,RT @smilelalis: Que alguien le explique a los ...,2020-11-16 20:08:45,Bogota,"RT Que alguien le explique a los uribistas, q...",[],[],[@smilelalis:]
4,RT @LaClavijo_: Colombia es al único país que ...,2020-11-16 20:08:37,Bogota,RT Colombia es al único país que lo toma por ...,[],[],[@LaClavijo_:]


In [16]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [17]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,RT @ELTIEMPO: 👉 Mientras la alerta por el paso...,2020-11-16 20:08:56,Bogota,RT 👉 Mientras la alerta por el paso del hurac...,[],[],[@ELTIEMPO:],[👉]
1,RT @IDEAMColombia: El huracán #IOTA ahora es d...,2020-11-16 20:08:47,Bogota,"RT El huracán ahora es de categoría 5, prese...",[],[#IOTA],[@IDEAMColombia:],[]
2,@rudolf_hommes @velascoluisf @teleislas Si en ...,2020-11-16 20:08:45,Bogota,"Si en condiciones relativamente ""normales"" ...",[],[],"[@rudolf_hommes, @velascoluisf, @teleislas]",[]
3,RT @smilelalis: Que alguien le explique a los ...,2020-11-16 20:08:45,Bogota,"RT Que alguien le explique a los uribistas, q...",[],[],[@smilelalis:],[]
4,RT @LaClavijo_: Colombia es al único país que ...,2020-11-16 20:08:37,Bogota,RT Colombia es al único país que lo toma por ...,[],[],[@LaClavijo_:],[]


In [21]:
data.shape

(100, 9)

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [18]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2020-11-16,100


In [24]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [22]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
0,👉,19
1,🚨,13
2,➡️,3
4,⚠️,2
3,🙏,1


In [25]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [27]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,1), max_features=100)
tfidf = tfidf_vect.fit_transform(data.clean.values)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc
temp

,acompañados,acuerda,adulto,ahora,alerta,alguien,altamar,andrés,aquí,archipiélago,...,vaina,van,vea,vez,vida,video,vientos,vía,zo,único
acompañados,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.438235,0.0,0.000000,0.000000
acuerda,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.822148
adulto,0.000000,0.000000,1.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000
ahora,1.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.438235,0.0,0.000000,0.000000
alerta,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.631837,0.000000,0.825689,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.898969,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
video,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.738249,0.000000,...,0.0,0.000000,1.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.000000
vientos,0.438235,0.000000,0.0,0.438235,0.000000,0.0,0.0,0.192148,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000
vía,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.0,0.000000,0.000000
zo,0.000000,0.000000,0.0,0.000000,0.898969,0.0,0.0,0.590987,0.000000,0.876312,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000


In [29]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)